# Google Gemini 2.5 Pro for Audio Understanding & Transcription




In [ ]:
!pip -q install google-genai jinja2

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_AI_STUDIO')

In [ ]:
import os
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

## Podcast Example


In [ ]:
from jinja2 import Template

# Generate a structured response using the Gemini API
prompt_template = Template("""Generate a transcript of the episode. Include timestamps and identify speakers.

Speakers are:
{% for speaker in speakers %}- {{ speaker }}{% if not loop.last %}\n{% endif %}{% endfor %}

eg:
[00:00] Brady: Hello there.
[00:02] Tim: Hi Brady.

It is important to include the correct speaker names. Use the names you identified earlier. If you really don't know the speaker's name, identify them with a letter of the alphabet, eg there may be an unknown speaker 'A' and another unknown speaker 'B'.

If there is music or a short jingle playing, signify like so:
[01:02] [MUSIC] or [01:02] [JINGLE]

If you can identify the name of the music or jingle playing then use that instead, eg:
[01:02] [Firework by Katy Perry] or [01:02] [The Sofa Shop jingle]

If there is some other sound playing try to identify the sound, eg:
[01:02] [Bell ringing]

Each individual caption should be quite short, a few short sentences at most.

Signify the end of the episode with [END].

Don't use any markdown formatting, like bolding or italics.

Only use characters from the English alphabet, unless you genuinely believe foreign characters are correct.

It is important that you use the correct words and spell everything correctly. Use the context of the podcast to help.
If the hosts discuss something like a movie, book or celebrity, make sure the movie, book, or celebrity name is spelled correctly.""")

# Define the speakers and render the prompt
speakers = ["John"] #put names is peakers are not mentioend
prompt = prompt_template.render(speakers=speakers)

print(prompt)

Generate a transcript of the episode. Include timestamps and identify speakers.

Speakers are:
- John

eg:
[00:00] Brady: Hello there.
[00:02] Tim: Hi Brady.

It is important to include the correct speaker names. Use the names you identified earlier. If you really don't know the speaker's name, identify them with a letter of the alphabet, eg there may be an unknown speaker 'A' and another unknown speaker 'B'.

If there is music or a short jingle playing, signify like so:
[01:02] [MUSIC] or [01:02] [JINGLE]

If you can identify the name of the music or jingle playing then use that instead, eg:
[01:02] [Firework by Katy Perry] or [01:02] [The Sofa Shop jingle]

If there is some other sound playing try to identify the sound, eg:
[01:02] [Bell ringing]

Each individual caption should be quite short, a few short sentences at most.

Signify the end of the episode with [END].

Don't use any markdown formatting, like bolding or italics.

Only use characters from the English alphabet, unless yo

## Uploading the MP3

In [ ]:

# path to the file to upload
file_path = "/content/HS4830417304.mp3" #Upload MP3

# Upload the file to the File API
uploaded_file = client.files.upload(file=file_path) #important to keep refence cz that what we shall need for ref

file_name = uploaded_file.name
print(file_name)

myfile = client.files.get(name=file_name)
print(myfile)

files/juo4b6r2flzc
name='files/juo4b6r2flzc' display_name=None mime_type='audio/mpeg' size_bytes=47648753 create_time=datetime.datetime(2025, 4, 6, 3, 14, 30, 897351, tzinfo=TzInfo(UTC)) expiration_time=datetime.datetime(2025, 4, 8, 3, 14, 30, 873053, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 4, 6, 3, 14, 30, 897351, tzinfo=TzInfo(UTC)) sha256_hash='ZjM1ZDlmMGFiOWZkODc4ZjVkNzlmNzkzYmNjZjg5NjEzNTM0Y2EyMDViZmJkYWJiYjQ2MTcwZmM3MDBhNGQ2NQ==' uri='https://generativelanguage.googleapis.com/v1beta/files/juo4b6r2flzc' download_uri=None state=<FileState.ACTIVE: 'ACTIVE'> source=<FileSource.UPLOADED: 'UPLOADED'> video_metadata=None error=None


In [ ]:

response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=[prompt, uploaded_file],
)

print(response.text)

[00:00] Shaan: There's five ways to make money in a system, in any system, there's like five roles and you can decide which one you're going to be in.
[00:08] Shaan: So like, uh, one is the logistics and labor.
[00:12] Shaan: So like let's take a house, uh, you're building a house.
[00:15] Shaan: The guy who's actually literally laying the bricks and out there sweating in the heat and building the actual home, sort of gets like paid the least but does the worst, does the most work, right?
[00:24] Shaan: Does the hardest work and does the most work in the process, right?
[00:27] Shaan: He's the logistics and labor.
[00:28] Shaan: He's actually moving the bricks around.
[00:31] Shaan: And you could choose to do that.
[00:32] Shaan: You could choose to be kind of a worker bee who's moving the bricks, but you're going to work the hardest and you're going to capture the least.
[00:37] Shaan: And life feels kind of unfair.
[00:38] Shaan: It's unfair that, you know, the real estate developer 

In [ ]:
import re
import datetime

def timestamp_to_seconds(ts_str):
    """
    Converts an HH:MM:SS or MM:SS timestamp string to total seconds.

    Args:
        ts_str (str): Timestamp string in HH:MM:SS or MM:SS format.

    Returns:
        int or None: Total seconds from midnight, or None if parsing fails.
    """
    try:
        # Remove milliseconds if present
        ts_str = ts_str.split('.')[0]
        # Split timestamp into parts
        parts = list(map(int, ts_str.split(':')))

        if len(parts) == 3: # HH:MM:SS format
            h, m, s = parts
            return h * 3600 + m * 60 + s
        elif len(parts) == 2: # MM:SS format
            m, s = parts
            return m * 60 + s
        else:
            # Invalid number of parts
            return None
    except (ValueError, AttributeError, IndexError):
        # Return None if parsing fails for any reason
        return None

def seconds_to_timestamp(total_seconds):
    """
    Converts total seconds to an HH:MM:SS timestamp string.
    (No changes needed here, always outputs full format)

    Args:
        total_seconds (int): Total seconds from midnight.

    Returns:
        str: Timestamp string in HH:MM:SS format.
    """
    if total_seconds is None or total_seconds < 0:
        total_seconds = 0 # Default to 0 if input is invalid or negative
    # Calculate hours, minutes, and seconds
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    # Format as HH:MM:SS with leading zeros
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

def process_transcript(input_text, max_segment_duration=30):
    """
    Processes transcript text to join lines based on speaker and time.

    Joins consecutive lines if the speaker is the same AND the time elapsed
    since the start of the current segment is within max_segment_duration.
    Starts a new segment if the speaker changes OR the time limit is exceeded.
    Includes non-dialogue lines (like [MUSIC]) as separate lines.

    Args:
        input_text (str): The raw transcript text (multiline string).
        max_segment_duration (int): Maximum duration in seconds for a single
                                     speaker's segment before forcing a new
                                     timestamp. Defaults to 30.

    Returns:
        str: The processed transcript text as a multiline string.
    """
    # Use splitlines() for more robust handling of different newline characters
    lines = input_text.strip().splitlines()
    output_lines = [] # List to store the processed output lines

    # Variables to keep track of the current segment being built
    current_segment_start_ts_str = None # Timestamp string of the segment start
    current_segment_start_seconds = None # Timestamp in seconds of the segment start
    current_speaker = None               # Speaker of the current segment
    current_text_parts = []              # List of text pieces in the current segment

    # --- UPDATED Regex ---
    # Capture timestamp [HH:MM:SS] or [MM:SS], speaker, and text content.
    # Made the HH: part optional using (?:...)?
    line_regex = re.compile(r'^\[((?:\d{2}:)?\d{2}:\d{2}(?:\.\d+)?)\]\s*([^:]+?):\s*(.*)$')
    # Breakdown of timestamp part: ((?:\d{2}:)?\d{2}:\d{2}(?:\.\d+)?)
    # (                    - Start capture group 1 (full timestamp)
    #  (?:               - Start non-capturing group for HH:
    #   \d{2}:            - Match two digits and a colon (HH:)
    #  )?                - End non-capturing group, make it optional
    #  \d{2}:\d{2}        - Match MM:SS (required)
    #  (?:\.\d+)?        - Optionally match milliseconds (non-capturing)
    # )                    - End capture group 1

    for i, line in enumerate(lines):
        line = line.strip() # Remove leading/trailing whitespace
        if not line:
            continue # Skip empty lines

        match = line_regex.match(line)

        # --- Debugging Prints (Keep commented unless needed) ---
        # print(f"\nProcessing Line {i+1}: {line}")
        # if match:
        #     _ts, _spk, _txt = match.groups()
        #     _sec = timestamp_to_seconds(_ts)
        #     print(f"  Parsed: ts={_ts}, speaker='{_spk.strip()}', text='{_txt.strip()}', seconds={_sec}")
        # else:
        #     print(f"  No Match: Treating as non-dialogue.")
        # print(f"  Current Segment State: speaker='{current_speaker}', start_ts='{current_segment_start_ts_str}', start_sec={current_segment_start_seconds}")
        # --- End Debugging Prints ---

        if not match:
            # --- Handle non-standard lines ---
            if current_speaker is not None:
                segment_text = ' '.join(filter(None, current_text_parts))
                # print(f"  Finalizing previous (due to non-match): [{current_segment_start_ts_str}] {current_speaker}: {segment_text}") # DEBUG
                output_lines.append(f"[{current_segment_start_ts_str}] {current_speaker}: {segment_text}")
                current_speaker = None
                current_text_parts = []
                current_segment_start_ts_str = None
                current_segment_start_seconds = None
            # print(f"  Adding non-dialogue line: {line}") # DEBUG
            output_lines.append(line)
            continue

        # --- Process standard dialogue lines ---
        ts_str, speaker, text = match.groups()
        speaker = speaker.strip()
        text = text.strip()
        # Use the updated function to parse timestamp
        current_seconds = timestamp_to_seconds(ts_str)

        if current_seconds is None:
             print(f"Warning: Skipping line {i+1} due to invalid timestamp format: {line}")
             continue

        # --- Logic to decide whether to start a new segment ---
        start_new_segment = False
        reason = "" # DEBUG
        if current_speaker is None:
            start_new_segment = True
            reason = "No active segment" # DEBUG
        elif speaker != current_speaker:
            start_new_segment = True
            reason = f"Speaker changed ('{speaker}' != '{current_speaker}')" # DEBUG
        # Ensure start_seconds is not None before comparison
        elif current_segment_start_seconds is not None and \
             current_seconds - current_segment_start_seconds > max_segment_duration:
            start_new_segment = True
            reason = f"Time limit exceeded ({current_seconds - current_segment_start_seconds}s > {max_segment_duration}s)" # DEBUG
        else:
            reason = "Continuing segment" # DEBUG

        # print(f"  Decision: {'Start new segment' if start_new_segment else 'Continue segment'}. Reason: {reason}") # DEBUG

        # --- Process based on the decision ---
        if start_new_segment:
            if current_speaker is not None:
                segment_text = ' '.join(filter(None, current_text_parts))
                # print(f"  Finalizing previous (due to new segment): [{current_segment_start_ts_str}] {current_speaker}: {segment_text}") # DEBUG
                output_lines.append(f"[{current_segment_start_ts_str}] {current_speaker}: {segment_text}")

            # Use seconds_to_timestamp to ensure consistent HH:MM:SS output format
            current_segment_start_ts_str = seconds_to_timestamp(current_seconds)
            current_segment_start_seconds = current_seconds
            current_speaker = speaker
            current_text_parts = [text]
            # print(f"  Starting new segment: speaker='{speaker}', start_ts='{current_segment_start_ts_str}', start_sec={current_segment_start_seconds}, text='{text}'") # DEBUG
        else:
            # Continue the current segment
            if text:
                # print(f"  Appending text: '{text}'") # DEBUG
                current_text_parts.append(text)
            # else: # DEBUG
                # print(f"  Skipping empty text append.") # DEBUG

    # --- Finalize the very last segment ---
    if current_speaker is not None:
        segment_text = ' '.join(filter(None, current_text_parts))
        # print(f"Finalizing last segment: [{current_segment_start_ts_str}] {current_speaker}: {segment_text}") # DEBUG
        output_lines.append(f"[{current_segment_start_ts_str}] {current_speaker}: {segment_text}")

    return "\n".join(output_lines)

In [ ]:
processed_transcript = process_transcript(response.text, max_segment_duration=30)

In [ ]:
processed_transcript

"[00:00:00] Shaan: There's five ways to make money in a system, in any system, there's like five roles and you can decide which one you're going to be in. So like, uh, one is the logistics and labor. So like let's take a house, uh, you're building a house. The guy who's actually literally laying the bricks and out there sweating in the heat and building the actual home, sort of gets like paid the least but does the worst, does the most work, right? Does the hardest work and does the most work in the process, right? He's the logistics and labor. He's actually moving the bricks around.\n[00:00:31] Shaan: And you could choose to do that. You could choose to be kind of a worker bee who's moving the bricks, but you're going to work the hardest and you're going to capture the least. And life feels kind of unfair. It's unfair that, you know, the real estate developer sitting in his second vacation home is going to make more off that house than the guy sweating outside every day, but that's th

In [ ]:
print(processed_transcript)

[00:00:00] Shaan: There's five ways to make money in a system, in any system, there's like five roles and you can decide which one you're going to be in. So like, uh, one is the logistics and labor. So like let's take a house, uh, you're building a house. The guy who's actually literally laying the bricks and out there sweating in the heat and building the actual home, sort of gets like paid the least but does the worst, does the most work, right? Does the hardest work and does the most work in the process, right? He's the logistics and labor. He's actually moving the bricks around.
[00:00:31] Shaan: And you could choose to do that. You could choose to be kind of a worker bee who's moving the bricks, but you're going to work the hardest and you're going to capture the least. And life feels kind of unfair. It's unfair that, you know, the real estate developer sitting in his second vacation home is going to make more off that house than the guy sweating outside every day, but that's the 

In [ ]:
PROMPT = """Make me a set of notes in the form of bullet points  \
(with time stamps at end of each idea (in the form HH:MM:SS)) to summarize this podcast \
The bullets should be based on the idea and don't need to be sequential.
Structure the ideas with a heading and subheadings. Don't include any prefix. here is transcript below :\n\n""" #

response = client.models.generate_content(
    model="gemini-2.5-pro-exp-03-25",
    contents=[PROMPT + processed_transcript],
)

print(response.text)

**Money, Happiness, and Wealth**

*   **The $50 Million Threshold**
    *   Billionaire Chamath Palihapitiya suggested on a podcast that significant life changes due to wealth plateau around $50 million, citing a (potentially nonexistent) study (00:02:06)
    *   Above $50 million, one supposedly has enough for homes, loved ones' security, and freedom from work (00:02:37)
    *   Shaan finds the common $70,000/year happiness threshold study to be nonsense (00:03:08)
    *   Jason Calacanis offered alternative wealth levels: $2M = good, $10M = FU money, $25M+ = dangerous (00:03:24)
    *   Sam jokes about the power of referencing non-existent studies to support a point (00:03:41)
    *   Sam believes the happiness number is context-dependent, not a fixed amount (00:04:21)
*   **Wealth Fluctuations & Mindset**
    *   Debate: Is it better to stay at a stable $100k or experience a rise to $100M and fall back to $100k? (00:04:36)
    *   Shaan prefers stability due to loss aversion and pot